<a href="https://colab.research.google.com/github/Jannatu37/codetopa-A/blob/main/Copy_of_waterQualityPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [71]:
import pandas as pd
import numpy as np

In [88]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [89]:
data = pd.read_csv("/content/water_potability (1).csv")
data

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
0,NaN,204.890455,20791.318981,7.300212,368.516441,564.308654,10.379783,86.990970,2.963135,0
1,3.716080,129.422921,18630.057858,6.635246,NaN,592.885359,15.180013,56.329076,4.500656,0
2,8.099124,224.236259,19909.541732,9.275884,NaN,418.606213,16.868637,66.420093,3.055934,0
3,8.316766,214.373394,22018.417441,8.059332,356.886136,363.266516,18.436524,100.341674,4.628771,0
4,9.092223,181.101509,17978.986339,6.546600,310.135738,398.410813,11.558279,31.997993,4.075075,0
...,...,...,...,...,...,...,...,...,...,...
3271,4.668102,193.681735,47580.991603,7.166639,359.948574,526.424171,13.894419,66.687695,4.435821,1
3272,7.808856,193.553212,17329.802160,8.061362,NaN,392.449580,19.903225,NaN,2.798243,1
3273,9.419510,175.762646,33155.578218,7.350233,NaN,432.044783,11.039070,69.845400,3.298875,1
3274,5.126763,230.603758,11983.869376,6.303357,NaN,402.883113,11.168946,77.488213,4.708658,1


In [91]:
#data = data.dropna()
data.isnull().sum()

,0
ph,491
Hardness,0
Solids,0
Chloramines,0
Sulfate,781
Conductivity,0
Organic_carbon,0
Trihalomethanes,162
Turbidity,0
Potability,0


**Imputation**

In [92]:
# Imputation is done here
# Imputing into null values
from sklearn.impute import SimpleImputer


imputer = SimpleImputer(strategy='median')
data = pd.DataFrame(imputer.fit_transform(data), columns=data.columns)


In [93]:
data['Potability'].value_counts()

,count
Potability,
0.0,1998
1.0,1278


**Resampling**

In [94]:
# Resampling is done here

In [95]:
X = data[['ph', 'Hardness', 'Solids', 'Chloramines', 'Sulfate', 'Conductivity', 'Organic_carbon', 'Trihalomethanes', 'Turbidity']]
y = data['Potability']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**Scaling**

In [96]:
# Scaling is done here
from sklearn.preprocessing import StandardScaler, MinMaxScaler


scaler = StandardScaler()
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [97]:
X_train

array([[0.47311525, 0.51891596, 0.27171856, ..., 0.48238685, 0.49079027,
        0.27120552],
       [0.47747855, 0.4821049 , 0.55255029, ..., 0.18584249, 0.50493346,
        0.4838104 ],
       [0.48829   , 0.52948183, 0.29079386, ..., 0.39439087, 0.53994235,
        0.65950789],
       ...,
       [0.47359742, 0.45374375, 0.49935618, ..., 0.39467646, 0.67841861,
        0.38087663],
       [0.62736912, 0.57901528, 0.2554584 , ..., 0.50077611, 0.3360708 ,
        0.53021533],
       [0.47843956, 0.49991248, 0.56112371, ..., 0.32617631, 0.50461516,
        0.4318777 ]])

In [98]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

In [99]:
# List of classifiers to evaluate
classifiers = {
   'Naive Bayes': MultinomialNB(),
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Decision Tree': DecisionTreeClassifier(),
    'Support Vector Classifier': SVC(),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'XGBoost': XGBClassifier(eval_metric='logloss')
}

In [100]:
from sklearn.metrics import accuracy_score

for name, model in classifiers.items():
    # Train the classifier
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Evaluate the classifier
    accuracy = accuracy_score(y_test, y_pred)
    print(f'{name}: {accuracy * 100:.2f}%')

Naive Bayes: 62.80%
Logistic Regression: 62.80%
Decision Tree: 57.62%
Support Vector Classifier: 69.21%
K-Nearest Neighbors: 63.41%
Random Forest: 68.14%
Gradient Boosting: 66.92%
XGBoost: 65.40%


**Feature Selection**

In [101]:
# Feature selection is done here

# Feature Selection - Forward Selection
from sklearn.model_selection import cross_val_score


selected_features = []
model = RandomForestClassifier()


num_features_to_select = 5


while len(selected_features) < num_features_to_select:
    best_score = -1
    best_feature = None


    for feature_idx in range(X.shape[1]):
        if feature_idx in selected_features:
            continue


        # Try adding the feature to the selected set
        candidate_features = selected_features + [feature_idx]


        # Evaluate the model's performance using cross-validation
        scores = cross_val_score(model, X.iloc[:, candidate_features], y, cv=5, scoring='accuracy')
        mean_score = np.mean(scores)


        # Keep track of the best-performing feature
        if mean_score > best_score:
            best_score = mean_score
            best_feature = feature_idx


    if best_feature is not None:
        selected_features.append(best_feature)
        print(f"Selected Feature {len(selected_features)}: {best_feature}, Mean Accuracy: {best_score:.4f}")

Selected Feature 1: 1, Mean Accuracy: 0.5562
Selected Feature 2: 2, Mean Accuracy: 0.5684
Selected Feature 3: 0, Mean Accuracy: 0.5867
Selected Feature 4: 4, Mean Accuracy: 0.6255
Selected Feature 5: 6, Mean Accuracy: 0.6297


**Threshold Moving**

In [ ]:
# Threshold moving is done here